In [1]:
#importing Libraries
import cv2
import numpy as np
import requests
import math
import serial
import time

In [2]:
#Open communication with bluetooth
ser = serial.Serial('COM3', 9600)

In [3]:
#Link camera with python
URL = "http://192.168.1.2"
AWB = True
cap = cv2.VideoCapture(URL + ":81/stream")
#cap.set(3,160)
#cap.set(4,120)

In [4]:
#Sending commands to arduino

def blueSend(MovDir):
    st= time.time()
    #ser.write(b'\xFF\x0C\x01')
    ser.write(bytes(MovDir,'utf_8'))
    et= time.time()
    elptime=et-st
 

In [5]:
#Camera Settings

def set_resolution(url: str, index: int=1, verbose: bool=False):
    try:
        if verbose:
            resolutions = "10: UXGA(1600x1200)\n9: SXGA(1280x1024)\n8: XGA(1024x768)\n7: SVGA(800x600)\n6: VGA(640x480)\n5: CIF(400x296)\n4: QVGA(320x240)\n3: HQVGA(240x176)\n0: QQVGA(160x120)"
            print("available resolutions\n{}".format(resolutions))

        if index in [10, 9, 8, 7, 6, 5, 4, 3, 0]:
            requests.get(url + "/control?var=framesize&val={}".format(index))
        else:
            print("Wrong index")
    except:
        print("SET_RESOLUTION: something went wrong")
        
        

def set_awb(url: str, awb: int=1):
    try:
        awb = not awb
        requests.get(url + "/control?var=awb&val={}".format(1 if awb else 0))
    except:
        print("SET_QUALITY: something went wrong")
    return awb



def set_quality(url: str, value: int=1, verbose: bool=False):
    try:
        if value >= 10 and value <=63:
            requests.get(url + "/control?var=quality&val={}".format(value))
    except:
        print("SET_QUALITY: something went wrong")


In [6]:
threshold1 = 30
threshold2 = 150
theta=0
r_width = 480
r_height = 640
minLineLength = 10
maxLineGap = 10
k_width = 5
k_height = 5
max_slider = 95
x=""
r=""
straightFlag=0
rightFlag=0
leftFlag=0
noLineFlag=0
rFlag=0;
rArr= []

In [7]:
if __name__ == '__main__':
    set_resolution(URL, index=8)

    while True:
        if cap.isOpened():
            
            ret, frame = cap.read()

            if ret:
                
                image = cv2.resize(frame,(r_width,r_height))
                # Convert the image to gray-scale
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                # given input image, kernel width =5 height = 5, Gaussian kernel standard deviation
                blurred = cv2.GaussianBlur(gray, (k_width, k_height), 0)
                # Find the edges in the image using canny detector
                edged = cv2.Canny(blurred, threshold1, threshold2)
                # Detect points that form a line
                lines = cv2.HoughLinesP(edged,1,np.pi/180,max_slider,minLineLength,maxLineGap)
                if lines is not None:
                    for x in range(0, len(lines)):
                        for x1,y1,x2,y2 in lines[x]:
                            if x1 != 0 or x2 != 0:
                                cv2.line(image,(x1,y1),(x2,y2),(255,0,0),3)
                                theta=theta+math.atan2((y2-y1),(x2-x1))
                                cx = (x1 + x2) / 2
                                cy = (y1 + y2) / 2
                                #cv2.circle(image,(cx,cy),5,(0,255,0),-1)
                                threshold=6
                                if cx >= 305:
                                    print("Turn right")
                                    x="R"
                                    r="Q"
                                    rArr.append(r)
                                    blueSend(x)
                                if(abs(theta)<threshold):
                                    print("On track")
                                    x="F"
                                    r="B"
                                    rArr.append(r)
                                    blueSend(x)
                                if cx <=190:
                                    print("Turn left")
                                    x="L"
                                    r="K"
                                    blueSend(x)
                                theta=0
                
                else:
                    print("No Lines")
                    if(noLineFlag==4):
                        noLineFlag=0
                        if len(rArr) > 4:
                            for i in range(len(rArr)-1, len(rArr)-5, -1):
                                blueSend(rArr[i])
                    else:
                        noLineFlag=noLineFlag+1
                    
                cv2.imshow("Line Detection",image)
                cv2.imshow("Edge",edged)
                if cv2.waitKey(1) & 0xff == ord('q'):
                    break
            else:
                break
    blueSend("S")
    cap.release()
    cv2.destroyAllWindows()

On track
Turn left
On track
Turn left
On track
On track
Turn left
Turn right
On track
On track
Turn left
Turn right
On track
Turn right
On track
On track
Turn left
On track
Turn right
On track
Turn right
On track
On track
Turn left
Turn right
On track
On track
On track
Turn left
On track
Turn left
On track
Turn left
On track
Turn left
On track
Turn left
On track
Turn left
On track
Turn left
On track
Turn left
On track
On track
Turn left
On track
On track
Turn left
On track
Turn left
On track
On track
On track
Turn left
On track
Turn left
On track
Turn left
On track
On track
Turn left
On track
Turn left
On track
On track
On track
On track
Turn left
On track
On track
On track
On track
Turn left
On track
On track
Turn left
No Lines
No Lines
On track
Turn left
On track
Turn left
On track
On track
On track
Turn left
On track
On track
Turn left
On track
On track
Turn left
On track
On track
Turn left
On track
On track
Turn left
On track
On track
Turn left
On track
Turn left
On track
Turn left